In [8]:
# Import libraries
import pandas as pd
import sys
import os

In [9]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Change directory into data directory
data_dir = os.path.join(parent_dir, 'data')

In [10]:
sys.path.append(parent_dir)

In [11]:
#List all files in the parent directory
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

# Use '..' to refer to the parent directory
list_files('..')

../
    LICENSE
    requirements.txt
    README.md
    .gitignore
    package-lock.json
    package.json
    app.py
    db_config.py
    tabs/
        recommendation.py
        menu.py
        recipe.py
        __pycache__/
            recommendation.cpython-311.pyc
            menu.cpython-312.pyc
            recipe.cpython-311.pyc
            menu.cpython-311.pyc
    images/
        wines2.jpeg
        wines.jpg
        logo.png
    html/
        univariate.html
    __pycache__/
        db_config.cpython-312.pyc
        db_config.cpython-311.pyc
        config.cpython-311.pyc
    model/
        wine_rec.py
        __pycache__/
            wine_rec.cpython-311.pyc
            wine_rec.cpython-312.pyc
    venv/
        pyvenv.cfg
        bin/
            streamlit.cmd
            pyftsubset
            jupyter-run
            Activate.ps1
            streamlit
            dotenv
            pip-chill
            python3
            pip3.12
            ttx
            python
           

In [12]:
import db_config as cfg

# 1. Extract

In [13]:
df_product = pd.read_csv(os.path.join(data_dir, 'products.csv'), index_col=None)

In [14]:
df_product.head()

,item_desc,corp_item_brand_name,pim_item_class_desc,pim_item_sub_class_desc,state,flavor,pim_tasting_notes,alcohol_percentage,sweetness_level,bitterness_level,...,body,serving_temperature,vintage_year,grape_variety,region,price,food_pairing,aroma,mouthfeel,finish
0,-196 CKTL VOD A(DL/G/P) CAN 3/8PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,OTHER,NaN,12.437938,8,10,...,Light,47.499006,2019,Chardonnay,Sonoma,76.968522,Pasta,Fruity,Silky,Short
1,-196 CKTL VOD PEACH 12 CAN 6/4PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,PEACH,NaN,12.995617,8,3,...,Light,43.392326,2005,Chardonnay,Sonoma,21.302044,Grilled Chicken,Fruity,Silky,Long
2,-196 CKTL VOD DBL LEM 12 CAN 6/4PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,LEMON,Fresh lemon peel. Tart and light sweet with ch...,11.234420,7,6,...,Medium,57.216822,2017,Merlot,Sonoma,23.523335,Cheese,Floral,Velvety,Long
3,-196 CKTL VOD GRFRUIT 12 CAN 6/4P,-196,COCKTAILS,COCKTAILS-OTHER,TX,GRAPEFRUIT,NaN,5.381421,4,4,...,Full,47.878084,2007,Cabernet Sauvignon,Tuscany,67.868299,Pasta,Fruity,Velvety,Short
4,10 CANE RUM 80,10 CANE,RUM,GOLD RUM,TX,NaN,NaN,13.381235,7,4,...,Full,51.834639,2008,Cabernet Sauvignon,Tuscany,95.581735,Pasta,Fruity,Silky,Short


# 2. Transform

In [15]:
df_product.dtypes

item_desc                   object
corp_item_brand_name        object
pim_item_class_desc         object
pim_item_sub_class_desc     object
state                       object
flavor                      object
pim_tasting_notes           object
alcohol_percentage         float64
sweetness_level              int64
bitterness_level             int64
acidity_level                int64
tannin_level                 int64
body                        object
serving_temperature        float64
vintage_year                 int64
grape_variety               object
region                      object
price                      float64
food_pairing                object
aroma                       object
mouthfeel                   object
finish                      object
dtype: object

# 3. Load

In [16]:
import pyodbc
import psycopg2

In [17]:
def get_column_types(df):
    # Map Pandas data types to SQL Server data types
    type_map = {
        'object': 'NVARCHAR(MAX)',
        'int64': 'BIGINT',
        'float64': 'DECIMAL(18, 5)',  # Use DECIMAL with precision and scale
        'datetime64[ns]': 'DATETIME2',
        'bool': 'BIT'
    }
    return [type_map.get(str(dt), 'NVARCHAR(MAX)') for dt in df.dtypes]


In [26]:
#Test connection, test insert table, test print table
def print_table(table_name, schema_name='dbo', limit=10):
    query = sql.SQL("SELECT * FROM {}.{} LIMIT %s").format(
        sql.Identifier(schema_name),
        sql.Identifier(table_name)
    )
    
    rows = execute_query(query, (limit,))
    
    if rows is not None:
        # Fetch column names
        column_query = sql.SQL("SELECT * FROM {}.{} LIMIT 0").format(
            sql.Identifier(schema_name),
            sql.Identifier(table_name)
        )
        column_result = execute_query(column_query)
        
        if column_result is not None and len(column_result) > 1:
            column_names = [desc[0] for desc in column_result[1].description]
        else:
            print(f"Unable to fetch column names. Using generic column names.")
            column_names = [f"Column_{i}" for i in range(len(rows[0]))]
        
        print(f"Contents of {schema_name}.{table_name} (first {limit} rows):")
        print(tabulate(rows, headers=column_names, tablefmt="grid"))
    else:
        print(f"No data returned from {schema_name}.{table_name}. The table might be empty or doesn't exist.")

# Create and populate the table
create_and_populate_table()

# Print the table contents
print_table('customers')

Table created and populated with sample data.
Unable to fetch column names. Using generic column names.
Contents of dbo.customers (first 10 rows):
+------------+---------------+---------------------+------------+
|   Column_0 | Column_1      | Column_2            |   Column_3 |
+============+===============+=====================+============+
|          1 | John Doe      | john@example.com    |         30 |
+------------+---------------+---------------------+------------+
|          2 | Jane Smith    | jane@example.com    |         28 |
+------------+---------------+---------------------+------------+
|          3 | Bob Johnson   | bob@example.com     |         35 |
+------------+---------------+---------------------+------------+
|          4 | Alice Brown   | alice@example.com   |         27 |
+------------+---------------+---------------------+------------+
|          5 | Charlie Davis | charlie@example.com |         32 |
+------------+---------------+---------------------+---------

In [19]:
def load_table(df, table_name, schema_name):
    # Define the connection parameters
    conn_params = {
        'host': cfg.HOST,
        'port': cfg.PORT,
        'database': cfg.DATABASE_NAME,
        'user': cfg.USER_ID,
        'password': cfg.USER_PASSWORD
    }

    # Create a psycopg2 connection using the connection parameters
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    # Drop the table if it already exists
    cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name}")
    conn.commit()

    # Create the table schema
    def get_postgres_column_types(df):
        type_map = {
            'object': 'TEXT',
            'int64': 'BIGINT',
            'float64': 'DECIMAL(18, 5)',
            'datetime64[ns]': 'TIMESTAMP',
            'bool': 'BOOLEAN'
        }
        return [type_map.get(str(dt), 'TEXT') for dt in df.dtypes]

    columns = ', '.join([f'"{col}" {dtype}' for col, dtype in zip(df.columns, get_postgres_column_types(df))])
    create_table_sql = f'CREATE TABLE {schema_name}.{table_name} ({columns})'
    cursor.execute(create_table_sql)
    conn.commit()

    # Clean up the float columns in the DataFrame
    for col in df.select_dtypes(include=['float']):
        df[col] = df[col].round(5)  # Round to 5 decimal places

    # Ensure there are no NaN or None values in numeric columns
    df.fillna(0, inplace=True)  # You can adjust the fill value as necessary

    # Insert the data into the table
    insert_sql = f'INSERT INTO {schema_name}.{table_name} VALUES ({",".join(["%s"] * len(df.columns))})'
    cursor.executemany(insert_sql, df.values.tolist())
    conn.commit()

    cursor.close()
    conn.close()
    print(f"DataFrame loaded successfully into {schema_name}.{table_name}")

In [ ]:
load_table(df_product, 'products', 'dbo')